<a href="https://colab.research.google.com/github/Omar-Aliii/AI-AGENT/blob/main/Question%20Answer%20Models/LLama2/Gradient/Instruction_finetuning_using_own_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradientai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.4/270.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 97.5 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.6.1
    Uninstalling pydantic-2.6.1:
      Successfully uninstalled pydantic-2.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
import os
os.environ['GRADIENT_ACCESS_TOKEN'] = "HEubR77OG4ETgOHwYQfd6VIcTLhvy7ai"
os.environ['GRADIENT_WORKSPACE_ID'] = "0f99a5d5-209f-43de-b5e4-dd07456cf183_workspace"

In [5]:
import pandas as pd
from gradientai import Gradient

def main():
    with Gradient() as gradient:
        base_model = gradient.get_base_model(base_model_slug="nous-hermes2")
        new_model_adapter = base_model.create_model_adapter(name="test model 3")
        print(f"Created model adapter with id {new_model_adapter.id}")

        sample_query = "### Instruction: what is the skin color of Luke Skywalker?\n\n### Response:"
        print(f"Asking: {sample_query}")

        # Before fine-tuning
        completion = new_model_adapter.complete(query=sample_query, max_generated_token_count=100).generated_output
        print(f"Generated (before fine-tune): {completion}")

        # Load the CSV data
        df = pd.read_csv('characters.csv')
        train_texts = df['name'].tolist()
        train_labels = df['skin_color'].tolist()

        samples = [{"inputs": text, "label": label} for text, label in zip(train_texts, train_labels)]

        # Fine-tuning
        num_epochs = 1
        batch_size = 8  # Adjust based on GPU memory
        total_samples = len(samples)
        batches = [samples[i:i + batch_size] for i in range(0, total_samples, batch_size)]

        for epoch in range(num_epochs):
            print(f"Fine-tuning the model, epoch {epoch + 1}")
            for batch in batches:
                new_model_adapter.fine_tune(samples=batch)

        # After fine-tuning
        completion = new_model_adapter.complete(query=sample_query, max_generated_token_count=100).generated_output
        print(f"Generated (after fine-tune): {completion}")

        new_model_adapter.delete()

if __name__ == "__main__":
    main()


Created model adapter with id ef74b97e-65f8-454f-95ef-28fc60dc70cd_model_adapter
Asking: ### Instruction: what is the skin color of Luke Skywalker?

### Response:
Generated (before fine-tune):  Luke Skywalker's skin color is not explicitly stated in the Star Wars films, but he is typically portrayed as having a fair or pale complexion. In the original Star Wars trilogy, Luke is played by Mark Hamill, who has a light complexion. In the prequel trilogy, which takes place before the original trilogy, Luke is portrayed as a young boy by Jake Lloyd, who also has a light complexion.
Fine-tuning the model, epoch 1
Generated (after fine-tune):  Luke Skywalker's skin color is white.
